# Project 2
# Rebalancing Strategy 

#### 2023-1학기
#### Version: 2023-05-16-v2

> 분반:
>
> 학번:
>
> 학과:
>
> 이름:

### To Do List

- rebalancing() 완성 (만약 BTC, ETH 외의 암호화폐를 추가적으로 고려하고 싶은 경우)
- 여러 자산 비율을 사용하여 성능 테스트


In [1]:
# 런타임이 종료되면 이 코드셀은 매번 다시 실행해야 합니다.

# pyupbit 설치
!pip install pyupbit

In [2]:
# Import

import numpy as np
import pandas as pd
import datetime

import pyupbit


In [5]:
# trading_helper 설치 및 버전 확인

!wget https://esohn.be/python/trading_helper.py -O trading_helper.py

import trading_helper
import importlib
importlib.reload(trading_helper)

print( f'Trading Helper Version: {trading_helper.TRADING_HELPER_VERSION}'  )

--2023-06-07 02:44:34--  https://esohn.be/python/trading_helper.py
Resolving esohn.be (esohn.be)... 18.64.8.58, 18.64.8.120, 18.64.8.126, ...
Connecting to esohn.be (esohn.be)|18.64.8.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6252 (6.1K) [text/x-python]
Saving to: ‘trading_helper.py’

trading_helper.py   100%[===================>]   6.11K  --.-KB/s    in 0.001s  

2023-06-07 02:44:35 (6.99 MB/s) - ‘trading_helper.py’ saved [6252/6252]

Trading Helper Version: 2023.05.15-v1


### AUTH_CODE는 반드시 자신의 Auth Code로 변경해야 합니다.

In [6]:
# ------------------------------------------------------------------------------------------------------
# https://esohn.be/account 의 AUTH CODE 로 실행 로그 기록
# ------------------------------------------------------------------------------------------------------

'''
AUTH_CODE = 'f587ea651febd5446efc61d5e3fef123'  -> 아래에 이렇게 auth code를 붙여넣으세요.
'''
AUTH_CODE = '🚨🚨🚨!!!이 부분 반드시 바꾸세요!!!🚨🚨🚨'

# Do not edit
# trading_helper.set_auth_code( AUTH_CODE )


In [7]:
# 가격정보 가져오기

from_date = '20220501'
to_date = '20230501'

df = trading_helper.get_multiple(from_date, to_date)


df.head()

,Unnamed: 0,index,price_btc,price_eth,price_doge,price_ada,bal_krw,bal_btc,bal_eth,bal_doge,...,bal2_ada,target_btc,target_eth,target_doge,target_ada,trade_btc,trade_eth,trade_doge,trade_ada,value
0,0,2022-05-01 09:00:00,49283000.0,3570000.0,166.0,996.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2022-05-02 09:00:00,49383000.0,3625000.0,170.0,1010.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2022-05-03 09:00:00,49501000.0,3667000.0,168.0,1000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,2022-05-04 09:00:00,48772000.0,3595000.0,167.0,997.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,2022-05-05 09:00:00,50631000.0,3752000.0,173.0,1140.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 아래 코드셀을 복사 해서 프로젝트 코드 페이지에 붙여넣으세요

In [8]:
# ------------------------------------------------------------------------------------------------------
# - 코드를 완성했다면 이 코드셀만 복사해서 competition 페이지 Rebalancing 탭에 라운드 별 코드 항목으로 붙여넣습니다.
# - 연습할 때는 다음과 같은 이유로 프로젝트 페이지의 에디터에서 직접 연습하지 말고 Colab에서 연습하세요. 
#   + Colab의 연습은 서버에 횟수가 기록되어 평가에 반영됩니다.
#   + Colab 노트북 자체에도 변경 history가 누적 됩니다. (부정행위 방지)
#   + 프로젝트 페이지에서 submit 횟수가 지나치게 빈번한 경우 submit이 일정시간 제한될 수 있습니다.
# ------------------------------------------------------------------------------------------------------

# Given Code Version 2023.05.16-v2

import numpy as np
import pandas as pd
import datetime

# rebalancing() 암호화폐 포트폴리오 리밸런싱 함수
'''
* ratio_  자산배분 비율 (총 합은 반드시 1.0이어야 함)
  투자하지 않는 암호화폐는 0으로 남겨둠
  
  예) 원화 10%, 비트코인(BTC) 50%, 이더리움(ETH) 40%
      ratio_krw = 0.1
      ratio_btc = 0.5
      ratio_eth = 0.4
      ratio_doge = 0.0
      ratio_ada = 0.0

  예) 원화 70%, 비트코인(BTC) 10%, 이더리움(ETH) 10%, 도지코인(DOGE) 5%, 에이다(ADA) 5%
      ratio_krw = 0.7
      ratio_btc = 0.1
      ratio_eth = 0.1
      ratio_doge = 0.05
      ratio_ada = 0.05

* threshold - 최소한으로 요구되는 총 자산 대비 변동 비율 (%)

  기본값 0: 조금이라도 변동될 때 매매
  예) 0.001: 전체 자산의 0.1% 변동시 매매, 0.005: 전체 자산의 0.5% 변동시 매매


* fee - 수수료, 기본값 0.001 (변경 없음)

* init_krw - 초기 투자금액, 기본값 100000000 (변경 없음)
'''

# Qualification 라운드 에서는 지정된 암호화폐 비율과 threshold 0.0을 사용해야 하며
# 그 외의 라운드에서는 암포화폐 비율과 threshold를 자유롭게 사용할 수 있음

# ------------------------------------------------
# EDITABLE (1)
#
DEFAULT_RATIO_KRW = 1.0
DEFAULT_RATIO_BTC = 0.0
DEFAULT_RATIO_ETH = 0.0
DEFAULT_RATIO_DOGE = 0.0
DEFAULT_RATIO_ADA = 0.0

DEFAULT_THRESHOLD = 0.0 # Qualification 라운드 에서는 이 값을 0.0으로
# ------------------------------------------------

def rebalancing( df, ratio_krw=DEFAULT_RATIO_KRW, ratio_btc=DEFAULT_RATIO_BTC, ratio_eth=DEFAULT_RATIO_ETH, ratio_doge=DEFAULT_RATIO_DOGE, ratio_ada=DEFAULT_RATIO_ADA, threshold=DEFAULT_THRESHOLD, fee=0.001, init_krw=100000000 ):

    # -------------------------------------------------------------------------------
    # DO NOT EDIT
    #
    ratio_sum = ratio_krw + ratio_btc + ratio_eth + ratio_doge + ratio_ada
    if abs(ratio_sum - 1.0) > 0.0001:
        print(f'The sum of the ratios must be 1. ({ratio_sum})')
        return
    # -------------------------------------------------------------------------------

    row_index = 0

    # 데이터프레임의 모든 행에 대해서 반복
    for row_index in range( len( df ) ):

        # 현재 데이터: cur_data
        cur_data = df.loc[ row_index, : ].copy()
        # 이전 데이터: prev_data
        if row_index > 0:
            prev_data = df.loc[ row_index-1, : ].copy()



        #------------------------
        # Balance 계산 (bal_)
        # 하루 전의 계산 후 잔고(bal2_)를 현재의 계산 전 잔고로 가져옴(bal_)
        if row_index == 0:
            # 첫째 날이라면 잔고는 krw만 존재
            cur_data[ 'bal_krw' ] = init_krw
        else:
            # 둘째 날 부터는 이전 날 계산 후 잔고(bal2_)를 현재 잔고(bal_)로 대입
            cur_data[ 'bal_krw' ] = prev_data[ 'bal2_krw' ]
            cur_data[ 'bal_btc' ] = prev_data[ 'bal2_btc' ]
            cur_data[ 'bal_eth' ] = prev_data[ 'bal2_eth' ]
            cur_data[ 'bal_doge' ] = prev_data[ 'bal2_doge' ]
            cur_data[ 'bal_ada' ] = prev_data[ 'bal2_ada' ]
            

        #------------------------
        # Total Value 계산 (value)
        # 전체 자산의 가치를 모두 합산
        cur_data[ 'value' ] = cur_data[ 'bal_krw' ]
        cur_data[ 'value' ] += cur_data[ 'price_btc' ] * cur_data[ 'bal_btc' ]
        cur_data[ 'value' ] += cur_data[ 'price_eth' ] * cur_data[ 'bal_eth' ]
        cur_data[ 'value' ] += 0 # EDITABLE (2)
        cur_data[ 'value' ] += 0 # EDITABLE (3)                                
                                

        #------------------------
        # Target 계산 (target_)
        # 비율에 따라 다시 조정되어야 할 목표 잔고 계산
        cur_data[ 'target_btc' ] = ratio_btc * cur_data[ 'value' ] / cur_data[ 'price_btc' ]
        cur_data[ 'target_eth' ] = ratio_eth * cur_data[ 'value' ] / cur_data[ 'price_eth' ]
        cur_data[ 'target_doge' ] = 0    # EDITABLE (4)
        cur_data[ 'target_ada' ] = 0     # EDITABLE (5)


        #------------------------
        # Trading Amount 계산 (trade_)
        # Target 잔고를 위해 매매해야 할 양을 계산
        cur_data[ 'trade_btc' ] = cur_data[ 'target_btc' ] - cur_data[ 'bal_btc' ]
        cur_data[ 'trade_eth' ] = cur_data[ 'target_eth' ] - cur_data[ 'bal_eth' ]
        cur_data[ 'trade_doge' ] = 0    # EDITABLE (6)
        cur_data[ 'trade_ada' ] = 0     # EDITABLE (7)


        #------------------------
        # 매매해야 할 양의 절대값이 총 자산의 일정 %를 넘지 않으면 매매하지 않음
        th_value = threshold * cur_data[ 'value' ]
        amount_btc = abs( cur_data[ 'trade_btc' ] * cur_data[ 'price_btc' ] )
        amount_eth = abs( cur_data[ 'trade_eth' ] * cur_data[ 'price_eth' ] )
        amount_doge = 0    # EDITABLE (8)
        amount_ada = 0     # EDITABLE (9)

        #--------------------------------------------------------------------------------------------------------------------
        # DO NOT EDIT
        # 계산 후 잔고 준비
        cur_data[ 'bal2_krw' ] = cur_data[ 'bal_krw' ]
        cur_data[ 'bal2_btc' ] = cur_data[ 'bal_btc' ]
        cur_data[ 'bal2_eth' ] = cur_data[ 'bal_eth' ]
        cur_data[ 'bal2_doge' ] = cur_data[ 'bal_doge' ]
        cur_data[ 'bal2_ada' ] = cur_data[ 'bal_ada' ]

        if amount_btc >= th_value:
            # BTC 잔고 변경 
            cur_data[ 'bal2_btc' ] += cur_data[ 'trade_btc' ]
            # krw 잔고 변경 (BTC 매매 반영 + 수수료 차감)
            cur_data[ 'bal2_krw' ] -= cur_data[ 'trade_btc' ] * cur_data[ 'price_btc' ] + amount_btc * fee

        if amount_eth >= th_value:
            # ETH 잔고 변경 
            cur_data[ 'bal2_eth' ] += cur_data[ 'trade_eth' ]
            # krw 잔고 변경 (ETH 매매 반영 + 수수료 차감)
            cur_data[ 'bal2_krw' ] -= cur_data[ 'trade_eth' ] * cur_data[ 'price_eth' ] + amount_eth * fee

        if amount_doge >= th_value:
            # DOGE 잔고 변경 
            cur_data[ 'bal2_doge' ] += cur_data[ 'trade_doge' ]
            # krw 잔고 변경 (DOGE 매매 반영 + 수수료 차감)
            cur_data[ 'bal2_krw' ] -= cur_data[ 'trade_doge' ] * cur_data[ 'price_doge' ] + amount_doge * fee

        if amount_ada >= th_value:
            # ADA 잔고 변경 
            cur_data[ 'bal2_ada' ] += cur_data[ 'trade_ada' ]
            # krw 잔고 변경 (ADA 매매 반영 + 수수료 차감)
            cur_data[ 'bal2_krw' ] -= cur_data[ 'trade_ada' ] * cur_data[ 'price_ada' ] + amount_ada * fee


        #--------------------------------------------------------------------------------------------------------------------
        # DO NOT EDIT
        #
        df.loc[ row_index, : ] = cur_data

        row_index += 1
        #--------------------------------------------------------------------------------------------------------------------




### 하나씩 비율을 조절하며 HPR, MDD 를 확인해 보는 방법

In [9]:
# 비트코인만 100%

rebalancing( df, ratio_krw=0.0, ratio_btc=1.0, ratio_eth=0.0, ratio_doge=0.0, ratio_ada=0.0, threshold= 0.0001 )

hpr, mdd = trading_helper.check_performance_rebal(df)

print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )


HPR=-20.8 MDD=58.4


In [ ]:
# 비트코인 60% 이더리움 40%

rebalancing( df, ratio_krw=0.0, ratio_btc=0.6, ratio_eth=0.4, ratio_doge=0.0, ratio_ada=0.0, threshold= 0.0001 )

hpr, mdd = trading_helper.check_performance_rebal(df)

print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )


### 비트코인, 이더리움, 도지코인, 에이다 그냥 가지고 있을 때와 리밸런싱 전략 4개의 HPR, MDD 비교

In [ ]:

rebalancing( df, ratio_krw=0.0, ratio_btc=1.0, ratio_eth=0.0, ratio_doge=0.0, ratio_ada=0.0 )
s_btc = df['value'].copy()
hpr, mdd = trading_helper.check_performance_rebal(df)
print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )

rebalancing( df, ratio_krw=0.0, ratio_btc=0.0, ratio_eth=1.0, ratio_doge=0.0, ratio_ada=0.0 )
s_eth = df['value'].copy()
hpr, mdd = trading_helper.check_performance_rebal(df)
print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )

rebalancing( df, ratio_krw=0.0, ratio_btc=0.0, ratio_eth=0.0, ratio_doge=1.0, ratio_ada=0.0 )
s_doge = df['value'].copy()
hpr, mdd = trading_helper.check_performance_rebal(df)
print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )

rebalancing( df, ratio_krw=0.0, ratio_btc=0.0, ratio_eth=0.0, ratio_doge=0.0, ratio_ada=1.0 )
s_ada = df['value'].copy()
hpr, mdd = trading_helper.check_performance_rebal(df)
print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )

rebalancing( df, ratio_krw=0.7, ratio_btc=0.1, ratio_eth=0.1, ratio_doge=0.05, ratio_ada=0.05 )
s1 = df['value'].copy()
hpr, mdd = trading_helper.check_performance_rebal(df)
print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )

rebalancing( df, ratio_krw=0.6, ratio_btc=0.2, ratio_eth=0.1, ratio_doge=0.05, ratio_ada=0.05 )
s2 = df['value'].copy()
hpr, mdd = trading_helper.check_performance_rebal(df)
print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )

rebalancing( df, ratio_krw=0.3, ratio_btc=0.2, ratio_eth=0.3, ratio_doge=0.1, ratio_ada=0.1 )
s3 = df['value'].copy()
hpr, mdd = trading_helper.check_performance_rebal(df)
print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )

rebalancing( df, ratio_krw=0.2, ratio_btc=0.2, ratio_eth=0.2, ratio_doge=0.2, ratio_ada=0.2 )
s4 = df['value'].copy()
hpr, mdd = trading_helper.check_performance_rebal(df)
print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )


In [ ]:

for i in np.arange(0, 0.6, 0.1):
    x = i
    y = 0.5 - i

    print( x, y, end=' ' )

    rebalancing( df, ratio_krw= 0.5, ratio_btc=0, ratio_eth=0, ratio_doge=x, ratio_ada=y )
    s4 = df['value'].copy()
    hpr, mdd = trading_helper.check_performance_rebal(df)
    print( f'HPR={hpr*100:.1f} MDD={mdd*100:.1f}' )


### 그래프 그려보기

In [ ]:

df2 = pd.DataFrame()
df2['BTC'] = s_btc
df2['ETH'] = s_eth
df2['DOGE'] = s_doge
df2['ADA'] = s_ada
df2['strategy1'] = s1
df2['strategy2'] = s2
df2['strategy3'] = s3
df2['strategy4'] = s4

df2.plot( figsize=(15,3) )

# 코드 보관 장소 
### 아래에 보관하고 싶은 코드를 자유롭게 수정/추가할 수 있습니다.
---

### Qual. Round 1,2 Code

### Comp. Round 1 (MDD 20) Code

### Comp. Round 2 (MDD 10) Code

### Bonus Round (MDD 30) Code